In [1]:
! pip install torch torchvision matplotlib pandas scikit-learn


In [6]:
import os
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import pandas as pd
from sklearn.model_selection import train_test_split
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from torchvision.datasets import ImageFolder

In [7]:
# Define the dataset class
class CoralReefDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.annotations = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, idx):
        img_path = os.path.join(self.root_dir, self.annotations.iloc[idx, 0])
        image = Image.open(img_path)
        label = 1 if self.annotations.iloc[idx, 1] == 'bleached' else 0

        if self.transform:
            image = self.transform(image)
        
        return image, label

In [8]:
# Image transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

In [9]:
# Load the training and validation datasets using ImageFolder
train_dataset = ImageFolder(root='train', transform=transform)
test_dataset = ImageFolder(root='test', transform=transform)

# Split the training dataset into training and validation sets
train_size = int(0.8 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_data, val_data = torch.utils.data.random_split(train_dataset, [train_size, val_size])

# Create DataLoaders for training, validation, and testing
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
val_loader = DataLoader(val_data, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Print some information about the datasets
print(f"Number of training samples: {len(train_data)}")
print(f"Number of validation samples: {len(val_data)}")
print(f"Number of test samples: {len(test_dataset)}")

Number of training samples: 590
Number of validation samples: 148
Number of test samples: 185


In [10]:
import torchvision.models as models

class CoralReefClassifier(nn.Module):
    def __init__(self):
        super(CoralReefClassifier, self).__init__()
        self.model = models.resnet18(pretrained=True)
        # Modify the final layer to match the number of output classes
        self.model.fc = nn.Linear(self.model.fc.in_features, 2)

    def forward(self, x):
        return self.model(x)

# Initialize the model, criterion, and optimizer
model = CoralReefClassifier()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Check if a GPU is available and move the model to GPU if possible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)


c:\Users\Lenovo\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Lenovo\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [11]:
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=5):
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0

        # Training loop
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        train_loss = running_loss / len(train_loader)
        train_acc = 100 * correct / total

        # Validation loop
        model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)

                val_loss += loss.item()
                _, predicted = torch.max(outputs, 1)
                val_total += labels.size(0)
                val_correct += (predicted == labels).sum().item()

        val_loss /= len(val_loader)
        val_acc = 100 * val_correct / val_total

        print(f"Epoch [{epoch+1}/{num_epochs}], "
              f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%, "
              f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%")

# Train the model
train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=10)


Epoch [1/10], Train Loss: 0.7921, Train Acc: 66.61%, Val Loss: 1.4461, Val Acc: 76.35%
Epoch [2/10], Train Loss: 0.4767, Train Acc: 75.08%, Val Loss: 0.7260, Val Acc: 72.30%
Epoch [3/10], Train Loss: 0.3549, Train Acc: 85.08%, Val Loss: 0.4596, Val Acc: 77.70%
Epoch [4/10], Train Loss: 0.2715, Train Acc: 88.98%, Val Loss: 0.4916, Val Acc: 82.43%
Epoch [5/10], Train Loss: 0.2394, Train Acc: 88.98%, Val Loss: 0.9139, Val Acc: 75.00%
Epoch [6/10], Train Loss: 0.2009, Train Acc: 91.02%, Val Loss: 0.5127, Val Acc: 78.38%
Epoch [7/10], Train Loss: 0.2166, Train Acc: 91.02%, Val Loss: 0.9212, Val Acc: 77.70%
Epoch [8/10], Train Loss: 0.2628, Train Acc: 88.64%, Val Loss: 0.6989, Val Acc: 81.08%
Epoch [9/10], Train Loss: 0.2029, Train Acc: 90.51%, Val Loss: 0.6427, Val Acc: 77.70%
Epoch [10/10], Train Loss: 0.1356, Train Acc: 95.76%, Val Loss: 0.6506, Val Acc: 79.05%


In [12]:
# Evaluation
model.eval()
test_correct = 0
test_total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        test_total += labels.size(0)
        test_correct += (predicted == labels).sum().item()

test_acc = 100 * test_correct / test_total
print(f"Test Accuracy: {test_acc:.2f}%")


Test Accuracy: 77.84%


In [13]:
# Function to make predictions on a single image
def predict_image(image_path, model, transform):
    # Load and preprocess the image
    image = Image.open(image_path)
    image = transform(image).unsqueeze(0)  # Add batch dimension
    
    # Move the image to the same device as the model
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    image = image.to(device)
    
    # Set the model to evaluation mode and make the prediction
    model.eval()
    with torch.no_grad():
        outputs = model(image)
        _, predicted = torch.max(outputs, 1)
    
    # Map predicted class index to human-readable label
    class_names = ['healthy', 'bleached']
    return class_names[predicted.item()]

In [19]:
# Example usage: Provide the path to an image for prediction
image_path = 'test/healthy/6685266501_befd5073d7_o.jpg'
predicted_label = predict_image(image_path, model, transform)
print(f"Predicted label: {predicted_label}")

Predicted label: healthy
